# Ensemble prediction of a precipitation extreme event

In [ ]:
# Activate the following in colab : 
!pip3 install netCDF4
!apt install proj-bin libproj-dev libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
    
import xarray as xr
import math
import numpy as np

In [ ]:
year =  '2020'
day  =  '03'
month = '02'

hour_1  ='18'

url='https://thredds.met.no/thredds/dodsC/meps25epsarchive/' +year+'/'+month+'/'+day+'/meps_extracted_2_5km_'+year+month+day+'T'+hour_1+'Z.nc'
forecast_1 = xr.open_dataset(url)


### Find a specific point and plot the forecast. 


In [ ]:
import math
import numpy as np

def findindex(alat,alon,plat,plon):
    #finding identic*l location of pos plat, plon in array alat[],alon[]
    abslat = np.abs(alat-plat)
    abslon = np.abs(alon-plon)
    c = np.maximum(abslon,abslat)
    latlon_idx = np.argmin(c)
    x, y = np.where(c == np.min(c))
    #print(alats[x,y],alon[x,y])
    x=int(x)
    y=int(y)
   
    return (x,y)

In [ ]:
[ix,jx] = findindex(forecast_1.latitude,forecast_1.longitude,69.649,18.956) # Tromsø

#### For example wind speed forecasts

In [ ]:
memb=1
accpp_point_forecast_1 = forecast_1.wind_speed[:,0,memb,ix,jx]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(forecast_1.time, accpp_point_forecast_1, '.-')

plt.title("Tromsø Wind Speed Forecast")
plt.show()

### Plot forecast on a map
Mean sea level pressure

In [ ]:
from mpl_toolkits.basemap import Basemap

timestep_1=20
mlevel=0
memb=7

fig = plt.figure(figsize=(7.2,7.2)) #11.7
plt.subplots_adjust(left=0.05, right=0.95, top=0.90, bottom=0.05, wspace=0.15, hspace=0.05)
ax = plt.subplot(111)
m = Basemap(projection='stere', 
            boundinglat=60, lon_0=10, lat_0=60.4, 
            resolution='l', 
            llcrnrlat= 64.0, urcrnrlat=74.0, llcrnrlon=-10.0, urcrnrlon=35.0)
 
m.drawcoastlines(color='#4D5D53'); m.fillcontinents(alpha=0.2)
parallels = np.arange(0.,90,10); m.drawparallels(parallels,labels=[1,1,1,0],fontsize=10)
meridians = np.arange(0.,360.,10); m.drawmeridians(meridians,labels=[0,0,0,1],latmax=80,fontsize=10)

x, y = m(np.array(forecast_1.longitude),np.array(forecast_1.latitude))               # compute map proj coordinates.

cs=m.contourf(x,y,np.array( forecast_1.air_pressure_at_sea_level[timestep_1,mlevel,memb,:,:]),cmap=plt.cm.coolwarm,extemd='max')

plt.title (np.datetime_as_string(forecast_1.time[timestep_1],unit='h'))
cbar = m.colorbar(cs,location='bottom',pad="5%")  
cbar.set_label('Mean sea level pressure')                  